In [4]:
import pandas as pd
#import argparse
import os

In [ ]:
# parser = argparse.ArgumentParser(description='choose_genes_cv')
# parser.add_argument('file_names', type=str, help='Name of folder and filenames for the promoters extracted')
# parser.add_argument('promoter_bedfile', type=str, help='Input location of promoter bedfile')
# parser.add_argument('Czechowski_rankedcv', type=str, help='Input location of Czechowski et al 2005 ranked cv dataset reanalysed by Will Nash')
# parser.add_argument('Mergner_rankedcv', type=str, help='Input location of Mergner et al 2020 ranked cv dataset')
# parser.add_argument('no_of_genes', type=int, help='Number of genes in each category to subset')
# parser.add_argument('Czechowski_gene_categories', type=str, help='Output location of microarray gene category subsets')
# parser.add_argument('Mergner_gene_categories', type=str, help='Output location of RNAseq gene category subsets')
# parser.add_argument('promoter_mapped_motifs', type=str, help='Input location of promoter mapped motifs bed file')
# parser.add_argument('promoters_filtered_contain_motifs', type=str, help='output location of the promoter bed file filtered so that each promoter contains at least one TFBS')
# parser.add_argument('Czechowski_allgenes', type=str, help='Output location of all filtered microarray genes')
# parser.add_argument('Mergner_allgenes', type=str, help='Output location of all filtered RNAseq genes')
# parser.add_argument('promoters_gff3', type=str, help='Input location of promoters gff3 file')
# parser.add_argument('option_remove_only5UTR', type=str, help='Optionally remove genes where only the Araport11 5UTR is present due to the promoter overlapping other genes',default = True, nargs="?")
# args = parser.parse_args()

In [32]:

file_names = 'non-overlapping_includingbidirectional_all_genes_newannotation'
#file_names = 'arabidopsis_selected_tissues'
#Czechowski_rankedcv = '../../data/genes/AtGE_dev_gcRMA__all_probes__CV.tsv'
Czechowski_rankedcv = f'../../data/output/{file_names}/genes/filtered80/AtGE_dev_gcRMA__all_probes__CV.tsv'
#Mergner_rankedcv = '../../data/genes/RNA_CVs.csv'
#promoter_bedfile = f'../../data/output/{file_names}/FIMO/promoters_5UTR.bed'
#promoterno5UTR_bedfile = f'../../data/output/{file_names}/promoters.gff3'
#Czechowski_gene_categories = f'../../data/output/{file_names}/genes/promoters_5UTR_czechowski_constitutive_variable_random_300.txt'
#Mergner_gene_categories = f'../../data/output/{file_names}/genes/promoters_5UTR_mergner_constitutive_variable_random_300.txt'
no_of_genes = 100
#promoter_mapped_motifs = f'../../data/output/{file_names}/FIMO/promoters_5UTR_motifs_mapped.bed'
#promoters_filtered_contain_motifs = f'../../data/output/{file_names}/FIMO/promoters_5UTR_filtered_contain_motifs.bed'
TF_list = '../../data/genes/Ath_TF_list.txt'
#Czechowski_allgenes = f'../../data/output/{file_names}/genes/promoters_5UTR_czechowski_allfilteredgenes.txt'

Czechowski_gene_categories_tfs = f'../../data/output/{file_names}/genes/tfs_czechowski_constitutive_variable_random_300'

In [44]:
def filter_genes_czechowski(TF_list, select_genes_file):
    """filter out genes from the microarray data which aren't in the promoter_bed"""
    #select_genes = pd.read_table(select_genes_file, sep='\t', header=0)
    select_genes = pd.read_table(select_genes_file, sep='\t', header=None)
    cols = ['rank','probe_id','AGI','expression_mean','expression_SD','expression_CV','proportion_of_values_present_in_mas5','presence_in_araport11','constitutive_in_araport11']
    select_genes.columns = cols
    #make AGI uppercase
    select_genes.AGI = select_genes.AGI.str.upper()

    #read in TF list
    TF_list_df = pd.read_table(TF_list, sep='\t',header=1)
    
    merged = pd.merge(TF_list_df, select_genes, left_on='Gene_ID',right_on='AGI', how='left')
    #remove NaNs in expression_CV column
    filtered_df = merged[merged.expression_CV.notnull()].copy()
    
    #sort by TF ID
    filtered_df.sort_values(['TF_ID'], inplace=True, ignore_index=True)
   
    #remove duplicates based on Gene_ID column
    filtered_df.drop_duplicates(subset=['Gene_ID'],inplace=True)
    
    #save df
    #filtered_df.to_csv(args.Czechowski_allgenes,sep='\t',columns=filtered_df.columns, index=False)
    
    #sort by CV value
    filtered_df.sort_values('expression_CV', inplace=True, ignore_index=True) 
    
    return filtered_df

In [ ]:
# def filter_genes_mergner(promoter_bed, select_genes_file):
#     """filter out genes from the RNA-seq data which aren't in the promoter_bed"""
#     select_genes = pd.read_csv(select_genes_file, header=0)
#     cols = ['AGI','transcription_class','transcription_family','expression_CV']
#     select_genes.columns = cols
#     #all present in Araport 11 column
#     select_genes['presence_in_araport11'] = 1
    
#     promoters = pd.read_table(promoter_bed, sep='\t', header=None)
#     col = ['chr','start','stop','AGI','dot1', 'strand','source','type','dot2','attributes']
#     promoters.columns = col

#     merged = pd.merge(promoters, select_genes, on='AGI', how='left')
#     #remove NaNs in expression_CV column
#     filtered_df = merged[merged.expression_CV.notnull()].copy()
    
#     #sort by chr and start
#     filtered_df.sort_values(['chr','start'], inplace=True, ignore_index=True)
    
#     #save df
#     filtered_df.to_csv(args.Mergner_allgenes,sep='\t',columns=filtered_df.columns,index=False)
    
#     #sort by CV value
#     filtered_df.sort_values('expression_CV', inplace=True, ignore_index=True)
#     return filtered_df 

In [30]:
def subSet_onCV(in_df, out_dir, no_of_genes):
    '''
    Extract the constitutive, variable, and control subsets based on CV values
    '''
    #filtering based on presence in the Araport11 annotation, define the first
    #n rows as the constitutive set and add label
    constitutive          = in_df[in_df.presence_in_araport11 == 1][0:no_of_genes]
    constitutive['state'] = 'constitutive'

    #define the last n rows as the variable set and add label
    variable          = in_df[in_df.presence_in_araport11 == 1][-no_of_genes:]
    variable['state'] = 'variable'

    #extract the rest of the rows as the control search space
    mid_range    = in_df[in_df.presence_in_araport11 == 1][(no_of_genes+1):-(no_of_genes+1)]

    #create 10 labelled bins
    mid_range['bins'] = pd.Series(pd.qcut(mid_range['expression_CV'], q = 10, precision = 2))

    #extract 10 random rows from these bins and label as the control set
    sample = no_of_genes/10
    sample_integar = int(str(sample).replace('.0', '')) #convert sample to an integar
    samples_from_bins = mid_range.groupby('bins').apply(pd.DataFrame.sample, sample_integar, random_state = 2)
    samples_from_bins['state'] = 'control'

    #concatenate and write as output
    output_set = pd.concat([constitutive[['AGI', 'state']], variable[['AGI', 'state']], samples_from_bins[['AGI', 'state']]], ignore_index = True)
    output_set.to_csv(out_dir, sep = '\t', index = False, header=False)
    
    #function from expressionVar_subsets_plot.py
    #__author__ = "Will Nash"
    # __copyright__ = "Copyright 2020, The Earlham Institute"
    # __credits__ = ["Will Nash", "Wilfried Haerty"]
    # __license__ = "GPL"
    # __version__ = "1.0"
    # __maintainer__ = "Will Nash"
    # __email__ = "will.nash@earlham.ac.uk"
    # __status__ = "Testing"
    #__modified_by__ "Sam Witham"

In [45]:
#make directory for the output files to be exported to
#dirName = f'{args.directory_path}/data/output/{args.file_names}'
dirName = f'../../data/output/{file_names}/genes'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " created") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")


filtered_czechowski = filter_genes_czechowski(TF_list,Czechowski_rankedcv)
#filtered_mergner = filter_genes_mergner(args.promoters_filtered_contain_motifs,args.Mergner_rankedcv)
#czechowksi subset
subSet_onCV(filtered_czechowski,Czechowski_gene_categories_tfs,no_of_genes)
#mergner subset
#subSet_onCV(filtered_mergner,args.Mergner_gene_categories,args.no_of_genes)

Directory  ../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/genes  already exists


In [38]:
# #make directory for the output files to be exported to
# #dirName = f'{args.directory_path}/data/output/{args.file_names}'
# dirName = f'../../data/output/{args.file_names}/genes'
# try:
#     # Create target Directory
#     os.mkdir(dirName)
#     print("Directory " , dirName ,  " created") 
# except FileExistsError:
#     print("Directory " , dirName ,  " already exists")


# filtered_czechowski = filter_genes_czechowski(args.promoters_filtered_contain_motifs,args.Czechowski_rankedcv)
# #filtered_mergner = filter_genes_mergner(args.promoters_filtered_contain_motifs,args.Mergner_rankedcv)
# #czechowksi subset
# subSet_onCV(filtered_czechowski,args.Czechowski_gene_categories,args.no_of_genes)
# #mergner subset
# #subSet_onCV(filtered_mergner,args.Mergner_gene_categories,args.no_of_genes)

In [46]:
filtered_czechowski

,TF_ID,Gene_ID,Family,rank,probe_id,AGI,expression_mean,expression_SD,expression_CV,proportion_of_values_present_in_mas5,presence_in_araport11,constitutive_in_araport11
0,AT4G26640.1,AT4G26640,WRKY,62.0,253983_at,AT4G26640,249.301390,47.634972,0.191074,98.0,1.0,0.0
1,AT5G45050.1,AT5G45050,WRKY,169.0,248973_at,AT5G45050,120.863167,25.667745,0.212370,98.0,1.0,0.0
2,AT5G09410.1,AT5G09410,CAMTA,176.0,245910_at,AT5G09410,389.393405,82.922480,0.212953,99.0,1.0,1.0
3,AT5G67580.1,AT5G67580,MYB_related,206.0,247012_at,AT5G67580,61.548600,13.354799,0.216980,100.0,1.0,0.0
4,AT4G38900.1,AT4G38900,bZIP,308.0,252969_at,AT4G38900,152.627983,34.909329,0.228722,99.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
565,AT3G16350.1,AT3G16350,MYB_related,11956.0,259379_at,AT3G16350,82.968393,242.133141,2.918378,81.0,1.0,0.0
566,AT2G40620.1,AT2G40620,bZIP,11963.0,255878_at,AT2G40620,123.417440,369.556407,2.994361,100.0,1.0,0.0
567,AT5G16540.1,AT5G16540,C3H,11972.0,250128_at,AT5G16540,357.916799,1102.500621,3.080327,98.0,1.0,0.0
568,AT1G26610.1,AT1G26610,C2H2,11989.0,260997_at,AT1G26610,30.842648,99.945493,3.240497,98.0,1.0,0.0
